# Triangular Arbitrage\n

In [ ]:
import requests, itertools
BASE='http://mock_apis:8000'
b = requests.get(f'{BASE}/api/binance/orderbook/BTCUSDT').json()
mb = (b['bids'][0][0]+b['asks'][0][0])/2
markets={('BTC','USDT'):mb,('ETH','USDT'):3700.0,('ETH','BTC'):3700.0/mb}
best=None
for a,b,c in itertools.permutations(['BTC','ETH','USDT'],3):
    if (a,b) in markets and (b,c) in markets and (c,a) in markets:
        cycle=markets[(a,b)]*markets[(b,c)]*markets[(c,a)]
        if not best or cycle>best[0]: best=(cycle,(a,b,c))
print(best)
